In [2]:
import os
from dotenv import load_dotenv
from tempfile import TemporaryFile
import rasterio
from pathlib import Path
import rasterio

load_dotenv()
os.chdir("/home/me/workspace/det_remota/trabalho_final")

In [3]:
RAW_FILES_PATH = Path("data/sentinel2/raw")
GRID_CELL_RANGE = range(0, 21)
YEAR_RANGE = range(2017, 2025)
MONTH_RANGE = [8]
LAYER_BANDS_DICT = {
    "TRUE_COLOR": [1,2,3],
    "FALSE-COLOR": [4,5,6],
    "MOISTURE": [7],
    "MSAVI2": [8],
    "NDWI": [9],
    "VEGETATION_INDEX": [10],
}

MERGED_CELLS_OUT_PATH = Path("data/sentinel2/preprocessed/merged_cells")

for grid_cell in GRID_CELL_RANGE:
    print(f"Processing Grid Cell {grid_cell}")
    for year in YEAR_RANGE:
        print(f"--Processing Year {year}")
        for month in MONTH_RANGE:
            print(f"----Processing Month {month}")

            out_path = MERGED_CELLS_OUT_PATH / f"{year}_{month}/{grid_cell}.tiff"
            if out_path.exists():
                print(f'----File {out_path} already exists. Skipping.')
                continue
            out_path.parent.mkdir(exist_ok=True, parents=True)

            raw_raster_dict = {
                layer: rasterio.open(RAW_FILES_PATH / f"{layer}/{year}/{month}/{grid_cell}.tiff")
                for layer in LAYER_BANDS_DICT.keys()
            }

            true_color = raw_raster_dict['TRUE_COLOR']
            profile = true_color.profile.copy()
            profile.update(count=10)
            
            with rasterio.open(out_path, 'w', **profile) as merged_raster:
                for layer, bands in LAYER_BANDS_DICT.items():
                    layer_raster = raw_raster_dict[layer]
                    layer_values = layer_raster.read(range(1, layer_raster.count))
                    merged_raster.write(layer_values, bands)
                merged_raster.write_mask(true_color.read(4))

                

Processing Grid Cell 0
--Processing Year 2017
----Processing Month 8
----File data/sentinel2/preprocessed/merged_cells/2017_8/0.tiff already exists. Skipping.
--Processing Year 2018
----Processing Month 8
----File data/sentinel2/preprocessed/merged_cells/2018_8/0.tiff already exists. Skipping.
--Processing Year 2019
----Processing Month 8
----File data/sentinel2/preprocessed/merged_cells/2019_8/0.tiff already exists. Skipping.
--Processing Year 2020
----Processing Month 8
----File data/sentinel2/preprocessed/merged_cells/2020_8/0.tiff already exists. Skipping.
--Processing Year 2021
----Processing Month 8
----File data/sentinel2/preprocessed/merged_cells/2021_8/0.tiff already exists. Skipping.
--Processing Year 2022
----Processing Month 8
----File data/sentinel2/preprocessed/merged_cells/2022_8/0.tiff already exists. Skipping.
--Processing Year 2023
----Processing Month 8
----File data/sentinel2/preprocessed/merged_cells/2023_8/0.tiff already exists. Skipping.
--Processing Year 2024
--

In [4]:
import random
import numpy as np
from skimage import exposure


def contrast_stretching(raster_values): 
    p2, p98 = np.percentile(raster_values, (2, 98))
    return exposure.rescale_intensity(raster_values, in_range=(p2, p98))

def min_max_stretching(raster_values): 
    p0, p100 = np.min(raster_values), np.max(raster_values)
    return exposure.rescale_intensity(raster_values, in_range=(p0, p100))

def adapt_hist(raster_values): 
    return exposure.equalize_adapthist(raster_values)


EQUALIZED_OUT_PATH = Path('data/sentinel2/preprocessed/equalized_cells')
EQUALIZE_BANDS_LIST = [
    (contrast_stretching, (1,2,3)),
    (contrast_stretching, (4,5,6)),
    (None, (7,)),
    (None, (8,)),
    (None, (9,)),
    (None, (10,)),
]

def equalize_cell(year: int, month: int, grid_cell: int, match_to = None):
    input_path = MERGED_CELLS_OUT_PATH / f"{year}_{month}" / f"{grid_cell}.tiff"
    output_path = EQUALIZED_OUT_PATH / f"{year}_{month}" / f"{grid_cell}.tiff"
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    print(f"Equalizing raster at {input_path}")
    
    with rasterio.open(input_path) as reader:
        profile = reader.profile.copy()
        with rasterio.open(output_path, 'w', **profile) as writer:
            for equalizer, bands in EQUALIZE_BANDS_LIST:
                print(f"--Equalizing bands {bands} using equalizer {equalizer}")
                if equalizer:
                    values = equalizer(reader.read(bands))
                else:
                    values = reader.read(bands)

                writer.write(values, bands)
                writer.write_mask(reader.read_masks())
        
        equalized = rasterio.open(output_path)
        
        if match_to:
            print("--Matching histograms to reference raster")
            matched = exposure.match_histograms(equalized.read(), match_to.read())
            with rasterio.open(output_path, 'w', **profile) as writer:
                writer.write(matched)
                writer.write_mask(equalized.read_masks())
            equalized = rasterio.open(output_path)

        print("--Done\n")
        return equalized


for year in YEAR_RANGE:
    for month in MONTH_RANGE:
        ref_cell = random.choice(GRID_CELL_RANGE)
        ref_raster = equalize_cell(year, month, grid_cell)

        for grid_cell in GRID_CELL_RANGE:
            if grid_cell == ref_cell:
                continue
            equalize_cell(year, month, grid_cell, ref_raster)

Equalizing raster at data/sentinel2/preprocessed/merged_cells/2017_8/20.tiff
--Equalizing bands (1, 2, 3) using equalizer <function contrast_stretching at 0x7c4cda367060>
--Equalizing bands (4, 5, 6) using equalizer <function contrast_stretching at 0x7c4cda367060>
--Equalizing bands (7,) using equalizer None
--Equalizing bands (8,) using equalizer None
--Equalizing bands (9,) using equalizer None
--Equalizing bands (10,) using equalizer None
--Done

Equalizing raster at data/sentinel2/preprocessed/merged_cells/2017_8/0.tiff
--Equalizing bands (1, 2, 3) using equalizer <function contrast_stretching at 0x7c4cda367060>
--Equalizing bands (4, 5, 6) using equalizer <function contrast_stretching at 0x7c4cda367060>
--Equalizing bands (7,) using equalizer None
--Equalizing bands (8,) using equalizer None
--Equalizing bands (9,) using equalizer None
--Equalizing bands (10,) using equalizer None
--Matching histograms to reference raster
--Done

Equalizing raster at data/sentinel2/preprocessed/me

In [5]:
import rasterio.merge

RAW_MOSAIC_OUT_PATH = Path('data/sentinel2/preprocessed/raw_mosaics')

for year in YEAR_RANGE:
    for month in MONTH_RANGE:
        print(f"Merging for period {year}-{month}")
        out_dir = RAW_MOSAIC_OUT_PATH / f"{year}_{month}.tiff"
        raster_readers = [
            rasterio.open(EQUALIZED_OUT_PATH / f"{year}_{month}" / f"{grid_cell}.tiff")
            for grid_cell in GRID_CELL_RANGE
        ]
        rasterio.merge.merge(
            raster_readers,
            use_highest_res=True,
            dst_path=out_dir
        )

Merging for period 2017-8
Merging for period 2018-8
Merging for period 2019-8
Merging for period 2020-8
Merging for period 2021-8
Merging for period 2022-8
Merging for period 2023-8
Merging for period 2024-8


In [6]:
import shutil
EQUALIZED_MOSAIC_OUT_PATH = Path('data/sentinel2/preprocessed/equalized_mosaics')

REF_YEAR = 2017

for month in MONTH_RANGE:
    ref_raster_path = RAW_MOSAIC_OUT_PATH / f"{REF_YEAR}_{month}.tiff"
    ref_raster_out_path = EQUALIZED_MOSAIC_OUT_PATH / f"{REF_YEAR}_{month}.tiff"
    shutil.copy(ref_raster_path, ref_raster_out_path)
    
    ref_raster = rasterio.open(ref_raster_out_path)
    for year in YEAR_RANGE:
        if year == REF_YEAR:
            continue
        
        print(f"Matching histograms for year {year}")

        year_raster_path = RAW_MOSAIC_OUT_PATH / f"{year}_{month}.tiff"
        year_raster = rasterio.open(year_raster_path)
        year_out_path = EQUALIZED_MOSAIC_OUT_PATH / f"{year}_{month}.tiff"

        profile = year_raster.profile.copy()
        matched = exposure.match_histograms(year_raster.read(), ref_raster.read())
        with rasterio.open(year_out_path, 'w', **profile) as writer:
            writer.write(matched)
            writer.write_mask(year_raster.read_masks())
        

Matching histograms for year 2018
Matching histograms for year 2019
Matching histograms for year 2020
Matching histograms for year 2021
Matching histograms for year 2022
Matching histograms for year 2023
Matching histograms for year 2024
